In [ ]:
import orchest
import pandas as pd
import time
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [ ]:
data = orchest.get_inputs()

In [ ]:
clarify_data = data['clarify_data']
energy_consumption_dfs = clarify_data['energy_consumption']
temperature_df = clarify_data['temperature']

et_curves_data = data['et_curves']
et_curves_day_of_week = et_curves_data['day_of_week_et_curves']

In [ ]:
start_looking_for_anomalies = datatime(2019, 12, 20)
